<a href="https://colab.research.google.com/github/MoronSlayer/Deep-Learning-Projects/blob/learner/KNN_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4, P4, or P100.

In [ ]:
!nvidia-smi

Sun Oct  2 15:52:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Setup:
Set up script installs
1. Updates gcc in Colab
1. Installs Conda
1. Install RAPIDS' current stable version of its libraries, as well as some external libraries including:
  1. cuDF
  1. cuML
  1. cuGraph
  1. cuSpatial
  1. cuSignal
  1. BlazingSQL
  1. xgboost
1. Copy RAPIDS .so files into current working directory, a neccessary workaround for RAPIDS+Colab integration.


In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 300 (delta 74), reused 99 (delta 55), pack-reused 171
Receiving objects: 100% (300/300), 87.58 KiB | 17.52 MiB/s, done.
Resolving deltas: 100% (136/136), done.
Traceback (most recent call last):
  File "rapidsai-csp-utils/colab/env-check.py", line 1, in <module>
    import pynvml
ModuleNotFoundError: No module named 'pynvml'


In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

In [ ]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:30
🔁 Restarting kernel...


In [ ]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

# RAPIDS is now installed on Colab.  You can copy your code into the cells below.  Enjoy!

In [ ]:
import cudf
import numpy as np
from cuml.datasets import make_blobs
from cuml.neighbors import NearestNeighbors as cuNearestNeighbors


class KNN_gpu:

    def __init__(self, n_neighbors):        
        self.n_neighbors = n_neighbors 	
	
    def GPU_knn_fit(self,input_Data):
      self.knn_cuml = cuNearestNeighbors()
      self.knn_cuml.fit(input_Data)      


    def GPU_knn_transform(self,test_data):      
      D_cuml, I_cuml = self.knn_cuml.kneighbors(test_data, n_neighbors = self.n_neighbors)
      return D_cuml,I_cuml

In [ ]:
from sklearn import neighbors
import numpy as np
from sklearn.neighbors import NearestNeighbors as skNearestNeighbors

class KNN_cpu:

    def __init__(self, n_neighbors):        
        self.n_neighbors = n_neighbors 	
	
    def CPU_knn_fit(self,input_Data):
      self.knn_sk = skNearestNeighbors()
      self.knn_sk.fit(input_Data)      


    def CPU_knn_transform(self,test_data):      
      D_cuml, I_cuml = self.knn_sk.kneighbors(test_data, n_neighbors = self.n_neighbors)
      return D_cuml,I_cuml

In [ ]:
knn = KNN_gpu(5)

In [ ]:
knn.GPU_knn_fit(x)

In [ ]:
D_cuml,I_cuml= knn.GPU_knn_transform(x)

In [ ]:
I_cuml

array([[    0, 19524, 23740, 39657, 25419],
       [    1, 37956, 18770, 21391, 15922],
       [    2,  3284,  3915, 31158,  5542],
       ...,
       [39997, 20358, 19207, 31152, 33920],
       [39998,  5186, 10857,  3598, 16728],
       [39999, 24322, 25183,  5201, 33328]])

In [ ]:
x = np.random.rand(40000, 20)
y = np.random.randint(2, size=x.shape[0])

In [ ]:
# data = '/content/dataset'
knn = KNN_gpu(x,5)

In [ ]:
knn

In [ ]:
knn_cuml = cuNearestNeighbors()
knn_cuml.fit(x)

NearestNeighbors()

In [ ]:
D_cuml, I_cuml = knn_cuml.kneighbors(x, n_neighbors=5)


In [ ]:
D_cuml.shape

(40000, 5)

# Next Steps #

For an overview of how you can access and work with your own datasets in Colab, check out [this guide](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92).

For more RAPIDS examples, check out our RAPIDS notebooks repos:
1. https://github.com/rapidsai/notebooks
2. https://github.com/rapidsai/notebooks-contrib